<a href="https://colab.research.google.com/github/DanRivaille/Perceptron-Multicapa/blob/master/src/notebooks/NeuronalNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Definición de la neurona
Se define la estructura de la neurona, la cual contendrá toda la información necesaria para que pueda realizar las operaciones que requiere.

In [1]:
import random

class Neuron:
  def __init__(self, function, value=0.0, id=None):
    self.value = value
    self.id = id
    self.funct = function
    self.der_funct = lambda F_x: F_x * (1 - F_x)
    self.next_layer = []
    self.prev_layer = []
    self.curr_weights = []
    self.bias = 0 # no yet implemented


  def add_layer(self, nodes_layer=None):
    self.prev_layer = nodes_layer
    
    for node in nodes_layer:
      node.next_layer.append(self)
      
    self.curr_weights = [random.random() for n in nodes_layer]


  def calculate_value(self, apply_function=True):
    sum = 0.0
    for (index, node) in enumerate(self.prev_layer):
      sum += node.value * self.curr_weights[index]

    self.value = sum + self.bias

    if apply_function:
      self.value = self.funct(self.value)

    return self.value

## Probando la neurona
Se realizan algunas pruebas para verificar que todas las funciones las realiza de una forma correcta.
  Se crea una neurona que use la función sigmoide como función de activación, luego se le agrega una capa a partir de los valores que deberian tener los nodos. Por ultimo se calcula el valor activado de la neurona.

In [2]:
import math

def sigmoide(x):
  return 1 / (1 + math.exp(-x))

values = [0.34, 0.99, 0.03]
nodes = [Neuron(sigmoide, v) for v in values]

n = Neuron(sigmoide)
n.add_layer(nodes)

n.calculate_value()

0.550154918047423

# Definición de la red neuronal
Se defina la estructura de la red neuronal, la cual tendra las funciones básicas para que pueda operar.

In [14]:
class NeuronalNetwork:
  def __init__(self, cant_input, cant_output, function):
    self.input_layer = [Neuron(function, id=(0, i + 1)) for i in range(cant_input)]
    self.output_layer = [Neuron(function, id=(-1, i + 1)) for i in range(cant_output)]
    self.hidden_layers = []

  def is_empty(self):
    return len(self.hidden_layers) == 0

  def get_function(self):
    return self.input_layer[0].funct

  def add_layers(self, layers):
    # Se obtiene la malla
    self.hidden_layers = self._make_connections(layers)

   # Se conecta el principio de la malla con la capa de entrada
    for node in self.hidden_layers[0]:
      node.add_layer(nodes_layer=self.input_layer)

    # Se conecta la capa de salida con el final de la malla
    for node in self.output_layer:
      node.add_layer(nodes_layer=self.hidden_layers[-1])


  def fit(self, X, y, learning_rate=0.1):
    # Por cada vector del conjunto de inputs, se ajusta el modelo
    for (i, x) in enumerate(X):
      print(x)

      self.predict(x)
      deltas = self._calculate_deltas(y[i])
      self._update_weights(deltas, learning_rate)

  
  def predict(self, input):
    # Se copian los valores del input en los nodos de la capa de entrada
    for (index, value) in enumerate(input):
      self.input_layer[index].value = value

    # Se calculan los valores de las capas ocultas
    for layer in self.hidden_layers:
      for node in layer:
        node.calculate_value()

    # Se calcula la salida sin aplicar la funcion de activacion al valor
    for node in self.output_layer:
      node.calculate_value()

    return [node.value for node in self.output_layer]


  def _make_connections(self, layers):
    '''
    Realiza las conexiones entre las capas ingresadas, y retorna una malla de 
    la red neuronal (sin entrada ni salida)
    '''
    nodes_mesh = self._create_mesh(layers)
    length = len(layers)

    for i in range(1, length):
      for node in nodes_mesh[i]:
        node.add_layer(nodes_layer=nodes_mesh[i - 1])

    return nodes_mesh


  def _create_mesh(self, layers):
    '''
    Crea una malla de capas, pero sin conecciones entre ellas
    '''
    nodes_mesh = []
    function = self.get_function()

    for current_layer in layers:
      new_nodes_layer = [Neuron(function, id=(current_layer, j + 1)) for j in range(current_layer)] # for debug
      #new_nodes_layer = [Neuron(function) for j in range(current_layer)]
      nodes_mesh.append(new_nodes_layer)

    return nodes_mesh


  def _update_weights(self, deltas, learning_rate):
    # Se actualizan los pesos de la primera capa oculta
    self._update_layer_weight(self.input_layer, 0, deltas, learning_rate)

    # Se actualizan los pesos de la segunda capa oculta
    for (i, layer) in enumerate(self.hidden_layers):
      self._update_layer_weight(layer, i + 1, deltas, learning_rate)


  def _update_layer_weight(self, layer, i_delta, deltas, learning_rate):
    for (i, node) in enumerate(layer):
      for (j, next) in enumerate(node.next_layer):
        new_weight = -learning_rate * deltas[i_delta][j] * next.value
        next.curr_weights[i] = new_weight


  def _calculate_deltas(self, y):
    deltas = self._create_delta_set()
    print('y:', y)
    
    # Se calculan los deltas de la capa de salida
    for (index, node) in enumerate(self.output_layer):
      deltas[-1][index] = (node.value - y[index]) * node.der_funct(node.value)

    length_hidden = len(self.hidden_layers)

    # Se calculan los deltas de las capas ocultas
    for i in range(length_hidden):
      i_lay = length_hidden - i - 1
      layer = self.hidden_layers[i_lay]

      for (j, node) in enumerate(layer):
        sum = 0.0

        for (k, next) in enumerate(node.next_layer):
          sum += deltas[i_lay + 1][k] * next.curr_weights[j]

        deltas[i_lay][j] = sum * node.der_funct(node.value)

    return deltas

  def _create_delta_set(self):
    deltas_hidden = [[0.0 for v in layer] for layer in self.hidden_layers]
    deltas_output = [0.0 for v in self.output_layer]

    return [*deltas_hidden, deltas_output]

## Probando la red neuronal

In [18]:
layers = [2, 3]

nn = NeuronalNetwork(3, 2, sigmoide)
nn.add_layers(layers)

In [19]:
X = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
y = [[1, 2], [2, 3], [3, 4]]

nn.fit(X, y)

[1, 2, 3]
y: [1, 2]
[4, 5, 6]
y: [2, 3]
[7, 8, 9]
y: [3, 4]


In [17]:
x = [1, 1, 6]
print(nn.predict(x))

[0.5118466136101689]


# Validacion de la red

## Funciones para comprobar el estado de la red en cada operacion
A continuacion se definen algunas funciones para comprobar el estado de la red, antes y despues de cada operacion que realiza, para verificar que este funcionando correctamente